# Quantification des incertitudes

## Résumé

Dans cette page, nous utilisons le cas de sollicitation d'une poutre en flexion dans le but d'identifier la loi qui s'ajuste à un échantillon unidimensionnel donné. Plus précisément, nous mettons en œuvre une démarche d'inférence statistique visant à identifier la distribution théorique sous-jacente.

# Introduction

On considère une poutre dont le module d'Young est noté $E$ (Pa). Le module d'Young $E$ caractérise la rigidité élastique du matériau constitutif de la poutre. Il relie la contrainte normale à la déformation longitudinale dans le domaine élastique linéaire.

<img src="poutre.png" width="200">

**Figure 1.** Déviation verticale $Y$ de la poutre de longueur $L$. 

Dans le cadre de cet exercice pédagogique, la distribution **réelle** (utilisée pour générer les données) est :

$$
\text{Beta}\left(\alpha = 0.9, \, \beta = 2.6, \, a = 2.8\times 10^7, \, b = 4.8\times 10^7\right)
$$

La loi précédente est la loi Beta généralisée, définie sur l'intervalle $[a,b]$ avec les paramètres de forme $\alpha$ et $\beta$.

Dans le cadre de cet exercice, on fait l'hypothèse que cette distribution est inconnue. On dispose d'un échantillon de 229 mesures de $E$ dans le fichier `sample_E.csv`. L'objectif de cet exercice est d'identifier la loi qui s'ajuste le mieux à cet échantillon. 

In [1]:
import openturns as ot

In [ ]:
data = ot.Sample.ImportFromCSVFile("sample_E.csv")
data.setDescription(["Young Modulus"])  # Optionnel
n = data.getSize()
print(f"Sample size={n}")

Sample size=229


In [3]:
data[0:10]

,Young Modulus
0,2.995319e+07
1,2.868042e+07
2,3.064115e+07
3,3.205095e+07
4,3.778573e+07
5,3.797256e+07
6,3.706933e+07
7,3.017982e+07
8,3.72172e+07
9,2.926821e+07


## Question 1 : statistiques descriptives

**Questions.**

- Calculer les principales caractéristiques de l'échantillon : moyenne, écart-type, médiane, minimum, maximum, intervalle inter-quartiles (c'est-à-dire à 25% et 75%).

**Indications méthodologiques :**  

Utiliser les méthodes `computeMean()`, `computeStandardDeviation()`, `computeQuantile()`, `getMin()` et `getMax()` de la classe `Sample`.

## Question 2 : histogramme empirique

**Questions.**
- Utiliser la classe `HistogramFactory` pour dessiner l'histogramme empirique de l'échantillon.

## Question 3 : ajuster une loi Beta

**Questions.**

* Utiliser la classe `BetaFactory` pour estimer les paramètres de la loi Beta s'ajustant aux données. 
* Dessiner la densité de probabilité de cette loi ajustée.
* Afficher les paramètres estimés.

## Question 4 : réaliser le test de Kolmogorov-Smirnov

**Questions.**

- Utiliser la classe `FittingTest.Lilliefors` pour réaliser le test de Lilliefors. Ce test constitue une adaptation du test de Kolmogorov-Smirnov tenant compte de l'incertitude liée à l'estimation des paramètres de la distribution à partir de l'échantillon (contrairement au test de Kolmogorov-Smirnov classique qui suppose des paramètres connus).
- Calculer la p-valeur.
- Doit-on rejeter la loi Beta ou non ?

## Question 5 : diagramme quantile-quantile

**Questions.**

- Utiliser la classe `VisualTest.DrawQQplot` pour dessiner le diagramme quantile-quantile. 
- L'ajustement est-il satisfaisant ?

## Question 6 : sélection de modèle par le critère BIC

On souhaite déterminer quelle loi s'ajuste le mieux au sens du critère BIC (c'est-à-dire menant à la plus petite valeur du critère BIC). 

**Questions.**

- Comparer les lois suivantes du point de vue du critère : uniforme, gaussienne, beta, log-normale. Quelle est la loi qui est la meilleure du point de vue du critère BIC, c'est-à-dire la loi la plus parcimonieuse ?
- La méthode `FittingTest.BestModelBIC` permet de renvoyer la distribution réalisant le meilleur critère BIC dans une liste de distributions candidates. Pour créer cette liste, on peut utiliser la méthode `GetContinuousUniVariateFactories`. Quelle distribution ressort de cette méthode ? Est-elle satisfaisante du point de vue du diagramme quantile-quantile ?

Veuillez noter que la méthode `ot.FittingTest.BIC` présente un comportement distinct suivant que l'argument d'entrée est une fabrique (`Factory`) ou une loi de probabilité (`Distribution`).

- Si une fabrique (`Factory`) est fournie, le nombre de paramètres estimés est alors égal au nombre de paramètres de la loi de probabilité.

- Si une distribution (`Distribution`) est fournie, le nombre de paramètres estimés doit alors être défini par l'utilisateur. Dans le cas contraire, la configuration par défaut fixe le nombre de paramètres estimés au nombre de paramètres de la distribution.

**Référence**

- https://github.com/openturns/openturns/issues/2198

## Question 7 : histogramme

**Questions.**

- Utiliser la classe `HistogramFactory` pour créer la distribution de type histogramme qui s'ajuste aux données. 
- Utiliser le diagramme quantile-quantile pour valider graphiquement cette distribution. 
- Quelles sont les limitations méthodologiques de l'approche par histogramme (sensibilité au nombre de classes, absence de lissage, discontinuités) ?
- L'histogramme constitue-t-il une distribution satisfaisante dans ce cas particulier ? Justifiez votre réponse.
- Dans quel type d'études industrielles ou académiques cette méthode serait-elle préférable aux ajustements paramétriques classiques ?

## Question 8 : méthode de lissage à noyau

**Questions.**

- Utiliser la classe `KernelSmoothing` pour utiliser une méthode à noyau pour estimer la distribution. 
- Utiliser le diagramme quantile-quantile pour valider le résultat graphiquement.
- Utiliser la méthode `setBoundaryCorrection()` pour activer la correction aux bords. Qu'observez-vous sur la qualité de l'ajustement dans les régions extrêmes de la distribution ?

**Remarque.**

La méthode `setBoundingOption()` permet de spécifier à quel bord la correction s'applique, c'est-à-dire à gauche, à droite ou aux deux bords.

## Question 9 : synthèse

On souhaite comparer les trois distributions suivantes :

- la loi Beta,
- la distribution issue de l'estimation par lissage à noyau,
- l'histogramme empirique.

**Questions.**

Comparer les distributions précédentes du point de vue de la densité de probabilité : 

- quelles différences observez-vous ?
- quelle semble être la loi la plus appropriée ?